In [1]:
import numpy as np
import pandas as pd

In [2]:
data_set = pd.read_csv('breast-cancer-wisconsin.csv',na_values=['?'])

In [3]:
data_set.head()

,id,clumpthickness,uniformcellsize,uniformcellshape,margadhesion,epithelial,barenuclei,blandchromatin,normalnucleoli,mitoses,benormal
0,1000025,5,1,1,1,2,1.0,3,1,1,2
1,1002945,5,4,4,5,7,10.0,3,2,1,2
2,1015425,3,1,1,1,2,2.0,3,1,1,2
3,1016277,6,8,8,1,3,4.0,3,7,1,2
4,1017023,4,1,1,3,2,1.0,3,1,1,2


class

2---benign
4---malignant


# DATA CLEANING

row deletion 
column deletion


In [4]:
# we will search and delete the rows whose class is classified other than 2 or 4


# there is data cleaning needed to this data set

# idhe maam

# there is ? in a column so set this value as a null

#data_set = data_set.replace({'?':np.nan})

In [5]:
# count the number of rows with NaN values in column 'B'
num_rows_with_nan = data_set['barenuclei'].isnull().sum()
num_rows_with_nan

16

In [6]:
#there are 16 rows with a null value in bare_nuclie column
# ill replace null value with the mean value of that column

mean_val = data_set['barenuclei'].mean()
mean_val

3.5446559297218156

In [7]:
data_set["barenuclei"] = data_set.barenuclei.fillna(int(mean_val))

In [8]:
data_set.isnull().sum()

id                  0
clumpthickness      0
uniformcellsize     0
uniformcellshape    0
margadhesion        0
epithelial          0
barenuclei          0
blandchromatin      0
normalnucleoli      0
mitoses             0
benormal            0
dtype: int64

In [9]:
# split the data into features and target

features = data_set.iloc[:,:-1].values
target = data_set.iloc[:,-1:].values


Split the data into training and testing sets

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

In [12]:
print(len(x_train),len(y_train))

489 489


In [13]:
print(len(x_test),len(y_test))

210 210


Data preprocessing


In [14]:
from sklearn.preprocessing import StandardScaler

In [15]:
# Perform feature scaling and normalization
sc = StandardScaler()
X_train = sc.fit_transform(x_train)
X_test = sc.transform(x_test)

# selecting the best hyperparameter using randomizzed search

In [16]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint as sp_randint

In [17]:
rf_classifier = RandomForestClassifier()

In [18]:
# Define the hyperparameter grid
param_dist = {'n_estimators': sp_randint(10, 100),
              'max_depth': sp_randint(1, 50),
              'max_features': ['sqrt', 'log2', 6],
              'min_samples_split': sp_randint(2, 20),
              'min_samples_leaf': sp_randint(1, 10),
              'criterion': ['gini', 'entropy']}

# Perform a randomized search with 5-fold cross-validation
random_search = RandomizedSearchCV(rf_classifier, param_distributions=param_dist,
                                   n_iter=100, cv=5, random_state=42, n_jobs=-1)

In [19]:
y_train=y_train.reshape(-1)

In [20]:
# Fit the randomized search object to the data
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000278FF822D60>,
                                        'max_features': ['sqrt', 'log2', 6],
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000278FCB15DC0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000278EBA8BF10>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000278FFD4D580>},
                   random_state=42)

In [21]:
# Print the best hyperparameters and the corresponding accuracy
print('Best hyperparameters:', random_search.best_params_)
print('Best accuracy:', random_search.best_score_)

Best hyperparameters: {'criterion': 'gini', 'max_depth': 39, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 15, 'n_estimators': 18}
Best accuracy: 0.9672417420576478


use the above generated hyperparameter for training 

In [22]:
# Train the random forest model
rf_classifier = RandomForestClassifier(n_estimators=98, max_depth=49, max_features='log2',criterion='gini',min_samples_leaf=4,min_samples_split=17)
rf_classifier.fit(X_train, y_train)


RandomForestClassifier(max_depth=49, max_features='log2', min_samples_leaf=4,
                       min_samples_split=17, n_estimators=98)

In [23]:
from sklearn.metrics import confusion_matrix

In [24]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [25]:
# Evaluate the model
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9714285714285714